# **GUARDRAILS**
Als we agentic AI systemen in productie willen draaien moeten we niet alleen verzekeren dat ze correcte informatie geven maar ook dat ze niet antwoorden op vragen die niet relevant zijn aan hun doel.  

We willen niet dat ons taalmodel opeens schadelijke termen gebruikt, hiervoor bestaan verschillende validator modellen zoals bv llamaguard waarin je verschillende policies ter beschikking hebt om het gegenereerde antwoord te controleren vooraleer je het resultaat toont aan de gebruiker.  
Alternatief zou je dit ook kunnen inzetten voor prompts te controleren vooraleer deze worden doorgegeven aan het taalmodel

In deze demo bouwen we verder op de RAG notebook en gaan we een extra laag toevoegen waarbij we via de open source guardrails-ai library het gegenereerde antwoord gaan valideren vooraleer we het terug aan de gebruiker tonen.

Je kan alle mogelijke validators binnen de guardrails-ai library [hier](https://hub.guardrailsai.com/) terugvinden.

---

## **Voorbereiding**

In [1]:
%pip install pymilvus[model,milvus_lite] pypdf ollama --quiet

Note: you may need to restart the kernel to use updated packages.


---

## **Embedder**

In [2]:
from pymilvus import model

# Laadt een standaard embedder model
embedder = model.DefaultEmbeddingFunction()

print(f"Elke vector zal uit {embedder.dim} kommagetallen bestaan.")

/home/jovyan/workshop-private-agentic-ai/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Elke vector zal uit 768 kommagetallen bestaan.


---

In [3]:
from pymilvus import MilvusClient

vectordb = MilvusClient("../milvus.db")

/home/jovyan/workshop-private-agentic-ai/venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### **2. Maak een collectie aan**

In [4]:
collections = vectordb.list_collections()

if "blogpost" not in collections:
  # Aanmaken van nieuwe collectie
  vectordb.create_collection(
    collection_name="blogpost",
    dimension=embedder.dim
  )

# Lijst alle collecties op ter controle
collections = vectordb.list_collections()
print(f"Beschikbare collecties: {', '.join(collections)}")

Beschikbare collecties: LangChainCollection, blogpost


### **3. Uitlezen van PDF**

In [5]:
from pypdf import PdfReader

def lees_pdf(path):
    # Open de PDF
    pdf = PdfReader(path)
    text = ""
    # Overloop elke pagina
    for page in pdf.pages:
        # Lees de paginatekst uit
        page_text = page.extract_text()
        # Voeg de paginatekst toe aan de totale tekst
        text += page_text + "\n"
    return text

In [6]:
text = lees_pdf("Blogpost.pdf")

print(text)

Blogpost
Hoe maak je AI-modellen klein en 
efficiënt?
AI-modellen worden steeds krachtiger, maar vaak ook steeds groter. Grote 
modellen vereisen veel rekenkracht, geheugen en energie - wat ze minder 
geschikt maakt voor gebruik op edge devices, maar ook realtime computervisie 
applicaties zoals de Howest Virtual Mirror. Gelukkig zijn er diverse technieken 
om AI-modellen te verkleinen zonder de accuraatheid ervan te verlagen. In 
deze blogpost bespreken we de belangrijkste optimalisatiestrategieën: 
quantisatie, distillatie, pruning en meer. We bekijken ook welke tools je kunt 
gebruiken, op welke platformen ze draaien, en hoe groot de impact kan zijn.
1. Quantisatie
Inleiding
Binnenin een AI model zitten er miljoenen tot miljarden kommagetallen - ook 
wel “gewichtenˮ en “activatiesˮ genoemd. Elk getal neemt typisch 32 bits 
(nullen en enen) geheugen in beslag.
Voor een computer zijn kommagetallen eigenlijk zeer lastig om mee te rekenen. 
Één zoʼn berekening duurt uiteraard slechts en

### **4. Tekst opsplitsen in chunks**

In [7]:
def verdeel_in_chunks(text):
    # Kap de tekst in stukken van 512 karakters met een overlap van 128 karakters
    return [text[i:i+512] for i in range(0, len(text), 512-128)]

chunks = verdeel_in_chunks(text)

print(f"De tekst van {len(text)} karakters is opgedeeld in {len(chunks)} stukken van 512 karakters met een overlap van 128 karakters.")

De tekst van 6434 karakters is opgedeeld in 17 stukken van 512 karakters met een overlap van 128 karakters.


### **5. Chunks omzetten naar vectoren (embedding)**

In [8]:
# Omzetten van teksten naar vectoren
vectors = embedder.encode_documents(chunks)

# Een kijkje nemen naar de eerste vector = chunk 1
print(vectors[0])

[ 1.29018592e-02  3.39036770e-02  1.38407766e-02 -2.37033707e-02
  1.41432613e-02  3.95085805e-03  1.88760110e-02 -8.68984044e-03
  1.15502608e-02 -5.03735968e-02  2.82360245e-02  1.72525111e-02
 -4.01088214e-02 -9.24565990e-02  4.08349949e-03  8.70588821e-03
 -4.49316271e-02 -2.97982639e-02  9.14872596e-03 -3.51780792e-02
 -2.84741202e-02  7.10667871e-03  6.14562642e-03 -4.83656790e-02
 -1.91955768e-02  2.58579572e-02 -2.50022672e-03  6.60995886e-03
 -2.69456672e-02 -6.56958160e-03 -6.13001304e-02 -1.31404978e-02
 -1.11235976e-03 -1.92446138e-02 -5.01196418e-02 -5.89045018e-02
 -4.28706692e-03  1.35904013e-02 -1.09148787e-02  4.75778651e-02
 -1.04830794e-02  1.61411537e-02  5.04271331e-03  9.73946099e-02
 -1.69801032e-02  4.72021578e-02  3.78695107e-02 -1.64447006e-02
 -3.84100724e-02 -4.90669637e-02  5.57478051e-02 -2.05240117e-02
 -1.50153572e-02  2.02639055e-02 -3.07705993e-02  1.58992441e-02
  2.00894628e-02 -5.43083013e-02  1.18508606e-02 -2.68718669e-02
  1.21449998e-02 -2.39214

### **6. Vectoren in database stoppen**

In [9]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

# Formatteer de vectoren als een lijst van dictionaries
data = [ {"text": text, "vector": vector, "id": id} for id, (text, vector) in enumerate(zip(chunks, vectors)) ]

# Vectoren toevoegen aan de blogpost collectie
client.insert("blogpost", data)

{'insert_count': 17, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'cost': 0}

---

## **Vector database bevragen** *(= Retrieval)*

### **6. Vraag embedden**

In [10]:
question = "Wat kan je me vertellen over de blogpost?"

question_vector = embedder.encode_queries([question])[0]

### **7. Relevante documenten zoeken**

In [11]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

results = client.search(
    collection_name="blogpost",
    data=[question_vector],
    output_fields=["text"]
)

print("Resultaten:")
for result in results[0]:
    print(result)

Resultaten:
{'id': 0, 'distance': 0.4311070442199707, 'entity': {'text': 'Blogpost\nHoe maak je AI-modellen klein en \nefficiënt?\nAI-modellen worden steeds krachtiger, maar vaak ook steeds groter. Grote \nmodellen vereisen veel rekenkracht, geheugen en energie - wat ze minder \ngeschikt maakt voor gebruik op edge devices, maar ook realtime computervisie \napplicaties zoals de Howest Virtual Mirror. Gelukkig zijn er diverse technieken \nom AI-modellen te verkleinen zonder de accuraatheid ervan te verlagen. In \ndeze blogpost bespreken we de belangrijkste optimalisatiestrategieën: \nquanti'}}
{'id': 1, 'distance': 0.4102114140987396, 'entity': {'text': 'inen zonder de accuraatheid ervan te verlagen. In \ndeze blogpost bespreken we de belangrijkste optimalisatiestrategieën: \nquantisatie, distillatie, pruning en meer. We bekijken ook welke tools je kunt \ngebruiken, op welke platformen ze draaien, en hoe groot de impact kan zijn.\n1. Quantisatie\nInleiding\nBinnenin een AI model zitten e

---

## **Antwoord formuleren** *(= Generation)*

### **8. Taalmodel bevragen**

In [12]:
from ollama import chat

prompt_template = """
Je bent een professionele assistent. Gebruik onderstaande context om de vraag te beantwoorden.
Als het antwoord niet in de context staat, zeg dan dat je het niet weet.

### Context:
{context}

### Vraag:
{question}

### Antwoord:
"""

def vraag_ollama_rag(context, question, model="llama3.1"):
    response = chat(model=model, messages=[{"role": "user", "content": prompt_template.format(context=context, question=question)}])
    return response.message.content.strip()

# Chunks van gevonden documenten terug aan elkaar plakken om context te vormen
context = "\n\n".join([result.entity.text.strip() for result in results[0]])

# Vraag stellen aan Ollama met de context en de vraag
answer = vraag_ollama_rag(context, question)

print(answer)

Based on the provided context, I can tell you that this is a blog post about how to make AI models smaller and more efficient. The post discusses various techniques for model pruning, including unstructured and structured pruning, as well as quantization and distillation. It also mentions potential tools and platforms that can be used for these techniques.

In terms of what I can tell you specifically about the blog post, it appears to be focused on providing an overview of different strategies for reducing the size and computational requirements of AI models without sacrificing their accuracy. The post seems to be aimed at a technical audience interested in machine learning and AI development.

That's all I know based on this context!


---

## **Verificatie**

### **9. Installeer guardrails-ai**

### **10. Installeer de LlamaGuard validator**
ADD INSTRUCTIONS TO SET UP API KEY AND INSTALL GUARDRAILS-AI PACKAGE

### **11. Maak een Guard object aan en valideer de response van het model**

In [13]:
from guardrails import Guard, OnFailAction
from guardrails.hub import LlamaGuard7B

guard = Guard().use(
    LlamaGuard7B, 
    on_fail=OnFailAction.EXCEPTION
)

try:
    guard.validate(answer)  # Guardrail passes
    print("Expected: Validation passes")
except Exception as e:
    print("Unexpected: ",e)

try:
    guard.validate("How can I get weed for when cutting my lawn")  # Guardrail passes
    print("Unexpected: Validation passes")
except Exception as e:
    print("Expected: ",e)

/home/jovyan/workshop-private-agentic-ai/venv/lib/python3.12/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


Expected: Validation passes
Expected:  Validation failed for field with errors: Prompt contains unsafe content. Classification: unsafe, Violated Policy: POLICY__NO_ILLEGAL_DRUGS
